# 1D Least-Square Inversion of Airborne Frequency Domain Surveys

**Based on:** Notebook by Marco Couto

Workflow reviewed to open real data and run single sounding l2 inversion

## Importing libraries

In [4]:
# SimPEG functionality
import simpeg.electromagnetics.frequency_domain as fdem
from simpeg.utils import plot_1d_layer_model, download, mkvc
from simpeg import (
    maps,
    data,
    data_misfit,
    regularization,
    optimization,
    inverse_problem,
    inversion,
    directives,
)

# discretize functionality
from discretize import TensorMesh

# Common Python functionality
import os
import numpy as np
import pandas as pd
from scipy.constants import mu_0
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets
import dill
%matplotlib inline
mpl.rcParams.update({"font.size": 12})

## For the paralellization
import time
from multiprocessing import Pool

write_output = False  # Optional

## Useful functions

In [6]:
#Function to plot tikhonov curves

def plot_tikhonov_curves(beta_values, phi_d, phi_m, phid_star=None, iteration=None, ax=None):
    ### This function is from Lindseýs class (EOSC 454/556B - Winter 2024), quite helpful.

    """
    Plot Tikhonov curves to visualize the trade-offs between data misfit (phi_d) and model norms (phi_m) 
    across a range of regularization parameters (beta_values).

    This function creates three subplots: 
    1. beta_values vs phi_d
    2. beta_values vs phi_m
    3. phi_m vs phi_d
    If a target phi_d value (phid_star) is provided, it is plotted as a dashed line in the first and third subplots.
    Specific iterations can be highlighted with markers if provided.

    Parameters:
        beta_values (array-like): Array of regularization parameter values (beta).
        phi_d (array-like): Array of data misfit values corresponding to each beta.
        phi_m (array-like): Array of model norm values corresponding to each beta.
        phid_star (float, optional): Target value of data misfit to plot as a reference line. Defaults to None.
        iteration (int, optional): Index of a specific iteration to highlight in the plots. Defaults to None.
        ax (array of matplotlib.axes.Axes, optional): Pre-existing axes for the plot. If None, axes will be created.
        
    Returns:
        ax (array of matplotlib.axes.Axes): The array of matplotlib axes with the plots.

    Notes:
        This function is from Lindsey's class (EOSC 454/556B - Winter 2024), and is designed to be a helpful
        visual tool in understanding the balance between fitting the data and regularization.

    Examples:
        >>> beta_values = np.logspace(-3, 1, 50)
        >>> phi_d = 1 / beta_values
        >>> phi_m = beta_values ** 2
        >>> plot_tikhonov_curves(beta_values, phi_d, phi_m, phid_star=0.1, iteration=25)
    """
    
    if ax is None: 
        fig, ax = plt.subplots(1, 3, figsize=(12,3))
    
    ax[0].plot(beta_values, phi_d)
    ax[1].plot(beta_values, phi_m)
    ax[2].plot(phi_m, phi_d)

    if phid_star is not None: 
        ax[0].plot(beta_values, np.ones_like(beta_values) * phid_star, "--k")
        ax[2].plot(phi_m, np.ones_like(beta_values) * phid_star, "--k")

    ax[0].set_ylabel("$\\phi_d$")
    ax[1].set_ylabel("$\\phi_m$")
    ax[2].set_ylabel("$\\phi_d$")
    ax[2].set_xlabel("$\\phi_m$")
    
    if iteration is not None: 
        ax[0].loglog(beta_values[iteration], phi_d[iteration], "C3o")
        ax[1].loglog(beta_values[iteration], phi_m[iteration], "C3o")
        ax[2].loglog(phi_m[iteration], phi_d[iteration], "C3o")

    for a in ax[:2]:
        a.invert_xaxis()
        a.set_xlabel("$\\beta$")
    plt.tight_layout()
    
    return ax

#Function to plot geographical trasmitter location
def plot_survey_lines(fdem_data, line_no, return_results=True, plot_results=True):
    """
    Plot the geographical distribution of transmitter locations for a specific survey line within a dataset.

    This function takes a line number, groups the data by line, and plots the transmitter locations for
    all lines in a faint scatter plot and highlights the transmitter locations for the specified line in a
    more pronounced scatter plot. The figure's axes are labeled with easting and northing in meters.

    Parameters:
        line_no (int or str): Identifier for the survey line whose data is to be plotted.
        display_results (bool, optional): If True, the function returns the data frame and station
                                          identifiers for the specified line. Defaults to False.

    Returns:
        tuple: Only if display_results is True. Returns a tuple containing:
               - pandas.DataFrame: Data frame for the specified line.
               - numpy.ndarray: Array of station identifiers for the specified line based on the fiducial code.

    Notes:
        This function assumes the existence of a globally accessible pandas DataFrame named `fdem_data` with
        at least the following columns: ['Line', 'fid', 'x_tx', 'y_tx']. It also assumes that the data has
        already been loaded and is available for processing.

    Examples:
        >>> plot_line_survey(101)
        >>> line_data, stations = plot_line_survey(101, display_results=True)
    """
    
    line_grouping = fdem_data.groupby('Line')
    line = line_grouping.get_group(line_no)
    line_stations = line.fid.values

    if plot_results:
        fig, ax = plt.subplots(figsize=(6, 6))        
        ax.scatter(fdem_data.x_tx, fdem_data.y_tx, s=0.5)
        ax.scatter(line.x_tx, line.y_tx, s=0.5)
        ax.set_xlabel('Easting (m)')
        ax.set_ylabel('Northing (m)')
        ax.ticklabel_format(axis='both', style='sci', scilimits=(6,5))
        ax.set_title(line_no)
        plt.tight_layout()

    if return_results:
        return line, line_stations

#Plot line of data
def plot_station_line_data(line, line_no, fid_list, freqs=[None], freqs_names=[None], display_results=False, plot_coaxial=False, plot_results=True):
    """
    Plot data for a specific survey station on a given line, including the geographical plot of the
    flight line and sounding data. This function creates several visualizations to analyze the frequency response
    at a specific station compared to the entire flight line data.

    Parameters:
        line_no (int or str): Identifier for the survey line whose data is to be plotted.
        fid (int or str): Identifier for the specific station (fiducial) on the flight line to highlight.
        display_results (bool, optional): If True, returns detailed data arrays for the specific station. Defaults to False.

    Returns:
        tuple: Only if display_results is True. Returns a tuple containing:
               - pandas.DataFrame: Data frame for the specific station.
               - int or str: Fiducial identifier.
               - list: In-phase data for the specific station across all frequencies.
               - list: Quadrature data for the specific station across all frequencies.

    Notes:
        Assumes availability of a globally accessible pandas DataFrame `fdem_data` with columns including 'Line', 'fid', 'x_tx', 'y_tx',
        and data fields formatted as 'cxi{frequency}' or 'cpi{frequency}' for in-phase data and 'cxq{frequency}' or 'cpq{frequency}'
        for quadrature data. The 'freqs' variable should be an iterable containing the frequency labels used in the data fields.
        This function also uses 'all_freqs_num', which should be the numerical values of frequencies for plotting.

    Examples:
        >>> plot_station_line_data(101, 205)
        >>> station_data, station_fid, inphase_data, quad_data = plot_station_line_data(101, 205, display_results=True)
    """


    #choose current line
    line_grouping = fdem_data.groupby('Line')
    line = line_grouping.get_group(line_no)

    #choose list of stations
    filtered_line = line[line["fid"].isin(fid_list)]
    
    # Select only numeric columns for mean calculation
    numeric_filtered_line = filtered_line.select_dtypes(include=["number"])
    
    # Group by 'fid' and calculate the mean for numeric columns only
    line_station = numeric_filtered_line.groupby("fid", as_index=False).mean()


    ### Getting the data for the sounding point and the data for the whole line
    inphase_station_data = []
    quad_station_data = []
    inphase_line_data = []
    quad_line_data = []


    for f in freqs_names:
        if plot_coaxial == True:
            inphase_station_data.append(line_station[f'cxi{f}'])
            quad_station_data.append(line_station[f'cxq{f}'])
            nphase_line_data.append(line[f'cxi{f}'])
            quad_line_data.append(line[f'cxq{f}'])
        else:
            inphase_station_data.append(line_station[f'cpi{f}'])
            quad_station_data.append(line_station[f'cpq{f}'])
            inphase_line_data.append(line_station[f'cpi{f}'])
            quad_line_data.append(line_station[f'cpq{f}'])

    inphase_station_data = np.array(inphase_station_data)
    quad_station_data = np.array(quad_station_data)
    inphase_line_data = np.array(inphase_line_data)
    quad_line_data = np.array(quad_line_data)
    
    if plot_results:

        ### Sounding plot
        fig, ax = plt.subplots(1, 2, figsize=(9, 5))
    
        ax[0].scatter(line.x_tx, line.y_tx, s=0.5)
        #if fid in line.fid.values:
        for i,fid in enumerate(fid_list):
            ax[0].scatter(line_station[line_station.fid == fid].x_tx, line_station[line_station.fid == fid].y_tx, s=40, marker='o', label = line_station[line_station.fid == fid].fid.values)
            ax[1].loglog(freqs, inphase_station_data[:,i], "-o", lw=2, , label = line_station[line_station.fid == fid].fid.values)
            ax[1].loglog(freqs, quad_station_data[:,i], ":o", lw=2, , label = line_station[line_station.fid == fid].fid.values)
        ax[0].set_xlabel('Easting (m)')
        ax[0].set_ylabel('Northing (m)')
        ax[0].ticklabel_format(style='sci')
        ax[0].set_title('Flight Line')
        ax[0].ticklabel_format(axis='both', style='sci', scilimits=(6,5))
        ax[0].legend()
    
        ax[1].grid(which="both")
        ax[1].set_xlabel("Frequency (Hz)")
        ax[1].set_ylabel("|Hs/Hp| (ppm)")
        ax[1].set_title("Sounding Data")
        ax[1].legend(["Real", "Imaginary"])
    
        fig.suptitle(f'{line_no} - Fiducial: {fid}', fontsize=14)
        plt.tight_layout()
        plt.show()

        ### Profile plot
        fig, ax = plt.subplots(2, 1, figsize=(9, 6))
        for i, f in enumerate(freqs):
            ax[0].semilogy(line_station.dist, inphase_line_data[i, :], "-o", lw=1, label = f"{freqs_names[i]}")
            ax[1].semilogy(line_station.dist, quad_line_data[i, :], "-o",  lw=1, label = f"{freqs_names[i]}")
        
        for i,fid in enumerate(fid_list):
            ax[0].axvline(line_station[line_station.fid == fid].dist.values[0], linestyle='--')
            #ax[0].text(line_station[line_station.fid == fid].dist.values[0], 0.1, line_station[line_station.fid == fid].fid.values, ha='center', va='bottom', color='black', fontsize=10)
            
            ax[1].axvline(line_station[line_station.fid == fid].dist.values[0], linestyle='--')
            #ax[1].text(line_station[line_station.fid == fid].dist.values[0], 0.1, line_station[line_station.fid == fid].fid.values, ha='center', va='bottom', color='black', fontsize=10)
            
        ax[1].set_xlabel("Profile Distance (m)")
        ax[0].set_ylabel("|Hs/Hp| (ppm)")
        ax[1].set_ylabel("|Hs/Hp| (ppm)")
        ax[0].legend()
        ax[1].legend()
        ax[0].set_title("Real")
        ax[1].set_title("Imaginary")
        ax[0].grid(which="both", linestyle="--", color='gray')
        ax[1].grid(which="both", linestyle="--", color='gray')
    
        # fig.suptitle(f'{line_no} - Fiducial: {fid}', fontsize=14)
        plt.tight_layout()
        plt.show()
        
    if display_results:
        return line_station, fid, inphase_station_data, quad_station_data

#Calculate the coordinates of transmitter and receiver
def calculate_tx_rx_coordinates(line, plot_histogram=False):
    """
    Calculates the transmitter (Tx) and receiver (Rx) coordinates for various frequencies
    based on azimuth values from the input dataframe.
    
    Parameters:
    - line (pd.DataFrame): Input dataframe containing x_tx and y_tx columns.
    - plot_histogram (bool): Whether to plot a histogram of azimuth values. Default is False.
    
    Returns:
    - pd.DataFrame: Updated dataframe with calculated Tx and Rx coordinates.
    """
    theta = []  # To store the co-azimuth in radians
    Az_rad = []  # To store the azimuth values in radians

    # Dictionary with Tx-Rx coil separation
    d = {
        'cp140k': 7.95,
        'cp40k': 7.93,
        'cp8200': 7.95,
        'cx3300': 9.06,
        'cp1800': 7.94,
        'cp400': 7.93
    }

    # Initialize arrays for Tx and Rx coordinates
    num_points = line.x_tx.values.shape[0]
    coords = {f"{key}_{suffix}": np.zeros(num_points) for key in d for suffix in ["tx_x", "tx_y", "rx_x", "rx_y"]}

    # Calculate coordinates based on azimuth
    for i in range(num_points - 1):
        deltay = line.y_tx.values[i + 1] - line.y_tx.values[i]
        deltax = line.x_tx.values[i + 1] - line.x_tx.values[i]
        # angle = np.arctan2(deltay, deltax)
        angle = np.arctan(deltay / deltax)
        theta.append(angle)
        # print(f'angle = {angle}')
        Az_rad.append(angle if angle >= 0 else 2 * np.pi + angle)

        for key, dist in d.items():
            dx = (dist / 2) * np.sin(angle)
            dy = (dist / 2) * np.cos(angle)

            coords[f"{key}_tx_x"][i] = line.x_tx.values[i] + dx
            coords[f"{key}_tx_y"][i] = line.y_tx.values[i] + dy
            coords[f"{key}_rx_x"][i] = line.x_tx.values[i] - dx
            coords[f"{key}_rx_y"][i] = line.y_tx.values[i] - dy

    Az_rad = np.array(Az_rad)
    Az_deg = Az_rad * 180 / np.pi

    if plot_histogram:
        plt.hist(Az_deg, bins=100)
        plt.title("Azimuth Distribution (Degrees)")
        plt.xlabel("Azimuth (degrees)")
        plt.ylabel("Frequency")
        plt.show()

        print(f'Mean azimuth: {np.mean(Az_deg):.2f} degrees')
        print(f'Std azimuth: {np.std(Az_deg):.2f} degrees')
        print('-----------------------------------\n')

    # Assign calculated Tx-Rx coordinates to the dataframe
    for key, values in coords.items():
        line[key] = values

    return line

#Calculate cumulative distances
def calculate_cumulative_distance(df, x_col='X', y_col='Y'):
    """
    Calculate the cumulative distance from the first point in a profile.

    Parameters:
    df (pd.DataFrame): The dataframe containing the X and Y coordinates.
    x_col (str): The name of the column containing the X coordinates. Default is 'X'.
    y_col (str): The name of the column containing the Y coordinates. Default is 'Y'.

    Returns:
    pd.DataFrame: The input dataframe with an additional column 'dist' containing the cumulative distances.
    """
    # Extract the X and Y coordinates
    x = df[x_col].values
    y = df[y_col].values
    
    # Calculate the differences between consecutive points
    dx = np.diff(x)
    dy = np.diff(y)
    
    # Calculate the Euclidean distance between consecutive points
    distances = np.sqrt(dx**2 + dy**2)
    
    # Prepend a 0 to the distances array to represent the distance from the first point to itself
    distances = np.insert(distances, 0, 0)
    
    # Calculate the cumulative distance
    cumulative_distances = np.cumsum(distances)
    
    # Add the cumulative distances to the dataframe
    df['dist'] = cumulative_distances
    
    return df

def make_directory(directory_name, absolute_path=False):
    """
    Checks if a directory exists one level above the current working directory.
    If it does not exist, the directory is created.

    :param directory_name: Name of the directory to check or create.
    :return: The relative path to the directory (e.g., "../my_directory").
    """
    # Get the current working directory
    work_directory = os.getcwd()

    # Get the parent directory (one level above)
    parent_directory = os.path.dirname(work_directory)

    # Create the full path for the directory
    directory_path = os.path.join(parent_directory, directory_name)

    # Check if the directory exists
    if not os.path.exists(directory_path):
        # Create the directory if it does not exist
        os.makedirs(directory_path)
        print(f"Directory '{directory_name}' created in '{parent_directory}'.")
    else:
        print(f"Directory '{directory_name}' already exists in '{parent_directory}'.")


    if absolute_path:
        # Return the absolute path (e.g., "/home/user/my_directory")
        return directory_path
    else:
        # Return the relative path (e.g., "../my_directory")
        return f"../{directory_name}"

SyntaxError: invalid syntax (1476016096.py, line 191)

In [ ]:
line_grouping = fdem_data.groupby('Line')
line = line_grouping.get_group(line_no)
line.fid.values.

## Inversion function

* Reading the data
* Defining Sources and Receivers
* Defining the data
* Halfspace inversion, pick 3 soundings, use this as reference model
* Inversion with Least Squares

## Importing the data

* Importing the data from database

In [ ]:
data_dir = '../database/'

csv_files = []
csv_titles = []
extension = "csv"

for root, dirs, files in os.walk(data_dir, topdown=False):
    for f in files:
        if f.split(".")[-1].upper() == extension.upper():
            csv_titles.append(f.split(".")[-2])
            csv_files.append(os.path.join(root, f))

csv_files

Read data from csv: 

In [ ]:
import os

# Get the current working directory (where the notebook is running)
current_dir = os.getcwd()

# Move one level up to the parent directory
parent_dir = os.path.dirname(current_dir)

# Construct the full path to the CSV file in the parent directory
csv_path = os.path.join(parent_dir, "block53_fdem_inv.csv")

fdem_data = pd.read_csv(csv_path) ## Set to the position of the csv_files list where the database you want is
pd.set_option('display.max_columns', len(fdem_data.columns))
fdem_data.head()

In [ ]:
fdem_data.rename(columns={'X': 'x_tx', 'Y':'y_tx', 'Z':'gpsz_tx'}, inplace=True) ## Change the name of the coordinates columns here, if needed:
np.array(fdem_data.columns) ## Shows all the columns names inside the dataframe

## Defining the system parameters

### System Frequencies

We define the system frequencies below:

In [ ]:
all_freqs_num = np.r_[135, 40, 8.2, 3.3, 1.8, 0.4]*1e3 ## Creates a numpy array with all frequencies
all_freqs_num[::-1].sort()
all_freqs_num

If you don`t want to invert coaxial frequencies, define them below: ?

In [ ]:
freq_cx = [3300] ## Coaxial frequencies - numpy array
all_freqs_num = np.delete(all_freqs_num, np.where(all_freqs_num == freq_cx)) ## Removes the coaxial frequencies from the list
all_freqs_num

Creating a list with the frequencies in string format too:

In [ ]:
all_freqs = ['140k', '40k', '8200', '3300', '1800', '400'] ## All frequencies
freq_cx_str = ['3300'] ## Coaxial frequencies - string array
all_freqs = np.array([f for f in all_freqs if f not in freq_cx_str]) ## Removes the coaxial frequencies from the list
all_freqs

### System geometry and physical parameters

In [ ]:
## Source orientation
source_orientation = ["z", "z", "z", "z", "z"] # "x", "y" or "z" - here we are using only the CP frequencies
moments = [17, 49, 72, 187, 359]  ## Tx dipole moments fromm Xcalibur´s report -  only cp geometry

## Receiver orientation
receiver_orientation = ["z", "z", "z", "z", "z"]  # "x", "y" or "z" - here we are using only the CP frequencies
data_type = 'ppm'  # "secondary", "total" or "ppm"

## Defining inversion parameters and testing on a specific sounding

In [ ]:
ipywidgets.interact(
    plot_survey_lines,
    fdem_data = ipywidgets.fixed(fdem_data),
    line_no = ipywidgets.SelectionSlider(
            options=fdem_data.Line.values,
            value=fdem_data.Line.values[0],
            description='Line: ',
            # disabled=False,
            # continuous_update=False,
            # orientation='horizontal',
            # readout=True
        ),
    return_results = ipywidgets.fixed(False),
    plot_resutls = ipywidgets.fixed(True)

)

# Plotting Line data

In [ ]:
line_no = 'L530100' ### "regular" flight line
lines = fdem_data['Line'].unique()
line, line_fids = plot_survey_lines(fdem_data, line_no, return_results=True, plot_results=True) ## Plots the map with the selected line for confirmation

In [ ]:
plot_station_line_data(fdem_data, line_no, [2490.8, 2490.5], freqs = all_freqs_num, freqs_names=all_freqs, display_results=False, plot_results=True)

# Choose the soundings

In [ ]:
line_grouping = fdem_data.groupby('Line')
line = line_grouping.get_group(line_no)

#Find all not nan fid values:
non_nan_fid = line[line['fid'].notna()]['fid'].values

#choose every 200 sounding of the last part 
list_fid = (non_nan_fid[3000:])[::100]


plot_station_line_data(fdem_data, line_no, list_fid, freqs = all_freqs_num, freqs_names=all_freqs, display_results=False, plot_results=True)

# Running the inversion for one sounding point

In [ ]:
#choose a single fid: from line
fid_n = list_fid[0]
line_station = line[line.fid == fid].iloc[0]


## Defining source and receiver data objects

In [ ]:
# Define inversion: parameters:
freqs = all_freqs_num
freqs_names = all_freqs
freq_avoid = [400., 1800., 8200.]
rm_real=True
rm_imag=True


#Define the Source orientation 
tx_orient = ["z", "z", "z", "z", "z"]
moments = [17, 49, 72, 187, 359]

# Define receiver orientation and type
rx_orient = ["z", "z", "z", "z", "z"] 
dtype = "ppm"

#Calculate the height (terrain clearance)
line_station.loc['height_tx'] = line_station['gpsz_tx'] - line_station['dtm']

######################################
### Defining Sources and Receivers ###
######################################

# Generate the labels with frequency values strings
source_locations = np.array([[
            getattr(line_station, f'cp{freq}_tx_x'),  # Transmitter X
            getattr(line_station, f'cp{freq}_tx_y'),  # Transmitter Y
            getattr(line_station, 'height_tx')  # Common height for the transmitter
            ] for freq in all_freqs])

receiver_locations = np.array([[
            getattr(line_station, f'cp{freq}_rx_x'),  # Transmitter X
            getattr(line_station, f'cp{freq}_rx_y'),  # Transmitter Y
            getattr(line_station, 'height_tx')  # Common height for the transmitter
            ] for freq in all_freqs])


#Create source and receiver list
source_list = []

for i, freq in enumerate(freqs):
 
    receiver_list = []

    #Add real readings
    if rm_real:
        if freq not in freq_avoid:
            receiver_list.append(
                fdem.receivers.PointMagneticFieldSecondary(
                    locations=np.c_[receiver_locations[i]].T,
                    orientation=rx_orient[i],
                    data_type=dtype,
                    component="real",
                )
            )
    else:
        receiver_list.append(
            fdem.receivers.PointMagneticFieldSecondary(
                locations=np.c_[receiver_locations[i]].T,
                orientation=rx_orient[i],
                data_type=dtype,
                component="real",
            )
        )
    if rm_imag:
            if freq not in freq_avoid:
                receiver_list.append(
                    fdem.receivers.PointMagneticFieldSecondary(
                        locations=np.c_[receiver_locations[i]].T,
                        orientation=rx_orient[i],
                        data_type=dtype,
                        component="imag",
                    )
                )
    else:
        receiver_list.append(
            fdem.receivers.PointMagneticFieldSecondary(
                locations=np.c_[receiver_locations[i]].T,
                orientation=rx_orient[i],
                data_type=dtype,
                component="imag",
            )
        )

    #Apprend source
    source_list.append(
            fdem.sources.MagDipole(
                receiver_list=receiver_list,
                frequency=freq,
                location=np.c_[source_locations[i]].T,
                orientation=tx_orient[i],
                moment=moments[i],
            )
        )

#########################
### Defining the data ###
#########################

## Defines the survey object
survey = fdem.survey.Survey(source_list)
survey.nD ## Check the number of Tx and Rx components (Real and Imaginary) - it should be 2 * the number of used frequencies

# Get arrays of data from sounding
inphase_data = []
quad_data = []

for f in freqs_names:
    inphase_data.append(line_station[f'cpi{f}'])
    quad_data.append(line_station[f'cpq{f}'])

inphase_data = np.array(inphase_data)
quad_data = np.array(quad_data)


#get rid of frequencies to avoid:
if rm_real and rm_imag:
    ind_avoid = [np.where(all_freqs_num == f)[0][0] for f in freq_avoid]
elif rm_real:
    ind_avoid = [np.where(all_freqs_num == f)[0][0] for f in freq_avoid]
elif rm_imag:
    ind_avoid = [np.where(all_freqs_num == f)[0][0] for f in freq_avoid]
else:
    ind_avoid = [None]

#Put everything in a single array

dobs = []
for i in range(len(inphase_data)):

    # if i == 3: ## In this analysis, we are not inverting the coaxial data
    #     pass
    # else:
    if rm_real and not rm_imag:
        if i not in ind_avoid:
            dobs = np.append(dobs, inphase_data[i])
        dobs = np.append(dobs, quad_data[i])
    elif rm_imag and not rm_real:
        dobs = np.append(dobs, inphase_data[i])
        if i not in ind_avoid:
            dobs = np.append(dobs, quad_data[i])
    elif rm_real and rm_imag:
        if i not in ind_avoid:
            dobs = np.append(dobs, inphase_data[i])
            dobs = np.append(dobs, quad_data[i])
    else:
        dobs = np.append(dobs, inphase_data[i])
        dobs = np.append(dobs, quad_data[i])


#Create data simpeg object:

# Defines the data object and assigns uncertainties with the noise floor
uncertainty_floor = 5.0e0
relative_error = 0.05 #What is relative error?
data_object = data.Data(survey, dobs=dobs, relative_error=relative_error, noise_floor=uncertainty_floor)


# Halspace Inversion

In [ ]:
### Mesh Definition for halfspace ###

layer_thick_halfspace = [10000]
n_layers_halfspace = 1

# Defining the mapping
log_conductivity_halfspace_map = maps.ExpMap(nP=n_layers_halfspace)

# Starting model is log-conductivity values (S/m)
starting_conductivity_model_hsp = np.log(1e-3 * np.ones(n_layers_halfspace))

# Reference model, same as starting 
reference_conductivity_model_hsp = starting_conductivity_model_hsp.copy()

### Forward simulation ###

simulation_hsp_L2 = fdem.Simulation1DLayered(
        survey=survey,
        thicknesses=[],
        sigmaMap=log_conductivity_halfspace_map
    )

### Data Misfit ###

dmis_hsp_L2 = data_misfit.L2DataMisfit(simulation=simulation_hsp_L2, data=data_object)

### Regularization ###

h = np.r_[layer_thick_halfspace]

# Create regularization mesh
regularization_mesh = TensorMesh([h], "N")

reg_L2 = regularization.WeightedLeastSquares(
        regularization_mesh,
        length_scale_x=10.0,
        # reference_model=np.r_[reference_conductivity_model, reference_conductivity_model[-1]],
        reference_model=reference_conductivity_model_hsp,
        reference_model_in_smooth=False
        )

#Set regularization parameters:
reg_L2.alpha_s = 1e-5 
reg_L2.alpha_x=1 

### Optimization ###
opt_L2 = optimization.InexactGaussNewton(
    maxIter=100, maxIterLS=20, maxIterCG=20, tolCG=1e-3
)

### Inversion ###
inv_prob_L2 = inverse_problem.BaseInvProblem(dmis_hsp_L2, reg_L2, opt_L2)

#Set inversion directives:
update_jacobi = directives.UpdatePreconditioner(update_every_iteration=True)
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=5)
beta_schedule = directives.BetaSchedule(coolingFactor=1.5, coolingRate=2)
target_misfit = directives.TargetMisfit(chifact=1.0)
save_L2_hp = directives.SaveOutputDictEveryIteration()

directives_list_L2 = [
    update_jacobi,
    starting_beta,
    beta_schedule,
    target_misfit,
    save_L2_hp
]

### Running the inversion ###
# Combine the inverse problem and the set of directives
inv_L2 = inversion.BaseInversion(inv_prob_L2, directives_list_L2)

# Run the inversion
# print(f"\n **** Running halfspace inversion for line {line_no} and station {stn}... ****")
recovered_halfspace_model_L2 = inv_L2.run(starting_conductivity_model_hsp)

## Get the recovered halfspace resistivity from model estimated
conductivities_hsp = log_conductivity_halfspace_map * recovered_halfspace_model_L2
resistivities_hsp = 1 / conductivities_hsp



In [ ]:
#Print halfspace results:
print(f"\n **** Halfspace inversion for line {line_no} and sounding {fid_n}... ****")
print("Resistivity halfspace: ", resistivities_hsp)

# Least Square Inversion

In [ ]:
### Mesh Definition ###

depth_min = 1                       # top layer thickness
depth_max = 205.                      # depth to lowest layer
geometric_factor = 1.1                # rate of thickness increase

#Increase layer thickness by geometric factor until depth_max
layer_thicknesses = [depth_min]
while np.sum(layer_thicknesses) < depth_max:
    layer_thicknesses.append(geometric_factor*layer_thicknesses[-1])

#Set number of layers
n_layers = len(layer_thicknesses) + 1  # Number of layers

### Defining the mapping ###
log_conductivity_map = maps.ExpMap(nP=n_layers)

### Starting model ##

# estimated host conductivity (S/m) from halfspace inversion
host_conductivity = 1 / resistivities_hsp[0]

# Starting model is log-conductivity values (S/m)
starting_conductivity_model = np.log(host_conductivity * np.ones(n_layers))

# Reference model is also log-resistivity values (S/m)
reference_conductivity_model = starting_conductivity_model.copy()

### Forward Simulation ###

simulation_L2 = fdem.Simulation1DLayered(
    survey=survey,
    thicknesses=layer_thicknesses,
    sigmaMap=log_conductivity_map
)

### Data Misfit ###

dmis_L2 = data_misfit.L2DataMisfit(simulation=simulation_L2, data=data_object)

### Regularization ###

# Define 1D cell widths
h = np.r_[layer_thicknesses, layer_thicknesses[-1]]
h = np.flipud(h)

# Create regularization mesh
regularization_mesh = TensorMesh([h], "N")

# Define regularization
reg_L2 = regularization.WeightedLeastSquares(
        regularization_mesh,
        # smallness=1e-5
        length_scale_x=10,
        reference_model=reference_conductivity_model,
        reference_model_in_smooth=False
    )

# Define regularization parameters:
reg_L2.alpha_s = 1e-5 # alpha_s
reg_L2.alpha_x=1 # alpha_x

### Optimization ###

opt_L2 = optimization.InexactGaussNewton(
    maxIter=100, maxIterLS=20, maxIterCG=20, tolCG=1e-3
)

### Inversion ###

inv_prob_L2 = inverse_problem.BaseInvProblem(dmis_L2, reg_L2, opt_L2)

#Inversion Directives
update_jacobi = directives.UpdatePreconditioner(update_every_iteration=False)
starting_beta = directives.BetaEstimate_ByEig(beta0_ratio=5)
beta_schedule = directives.BetaSchedule(coolingFactor=1.5, coolingRate=2)
target_misfit = directives.TargetMisfit(chifact=1.0)
save_L2 = directives.SaveOutputDictEveryIteration()

directives_list_L2 = [
    update_jacobi,
    starting_beta,
    beta_schedule,
    target_misfit,
    save_L2
]

### Inversion ###

# Combine the inverse problem and the set of directives
inv_L2 = inversion.BaseInversion(inv_prob_L2, directives_list_L2)

# Run the inversion
# print(f"\n **** Running layered inversion for line {line_no} and station {stn}... ****")
recovered_model_L2 = inv_L2.run(starting_conductivity_model)

resistivities_L2 = 1 / (log_conductivity_map * recovered_model_L2)